# Clase práctica 3: atributos visuales

**Profesor de Cátedra:** José M. Saavedra.

**Profesor Auxiliar:** Cristóbal Loyola (cloyola@dcc.uchile.cl)

Este tutorial muestra cómo ejecutar, en un entorno de Google Colab, el código del repositorio [jmsaavedrar/](https://github.com/jmsaavedrar//). El código permite realizar búsquedas por similitud a partir de capas ocultas de una red neuronal convolucional. Para ejecutarlo, también necesitaremos el repositorio base [jmsaavedrar/convnet2](https://github.com/jmsaavedrar/convnet2).

El primer paso será clonar los repositorios en el entorno de Colab:

In [35]:
!rm -rf convnet2
!git clone https://github.com/alvaro-neira/convnet2.git --branch tarea1
!rm -rf visual_attributes
!git clone https://github.com/alvaro-neira/visual_attributes.git

Cloning into 'convnet2'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 279 (delta 177), reused 161 (delta 84), pack-reused 0
Receiving objects: 100% (279/279), 77.21 KiB | 7.72 MiB/s, done.
Resolving deltas: 100% (177/177), done.
Cloning into 'visual_attributes'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 50 (delta 22), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [36]:
!pwd  # muestra el directorio actual
!ls   # muestra el contenido de este (archivos y directorios)

/content/data/dataset_atributos
color  convnet2  texture  visual_attributes


## Preparación de los datos

In [37]:
# Crear directorio para los datos 
!mkdir /content/data
!ls /content/

mkdir: cannot create directory ‘/content/data’: File exists
convnet2  data	sample_data  visual_attributes


In [38]:
# Descargar y descomprimir los datos
%cd /content/data
!rm -rf dataset_atributos
!wget https://www.dropbox.com/s/ri743kwqh8t6a7r/dataset_atributos.zip
!unzip -q dataset_atributos.zip
!rm dataset_atributos.zip
!ls

/content/data
--2021-11-12 02:48:32--  https://www.dropbox.com/s/ri743kwqh8t6a7r/dataset_atributos.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ri743kwqh8t6a7r/dataset_atributos.zip [following]
--2021-11-12 02:48:33--  https://www.dropbox.com/s/raw/ri743kwqh8t6a7r/dataset_atributos.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc107c41026598443c4b4b10d15c.dl.dropboxusercontent.com/cd/0/inline/BZwvAd2HPjDU39sa6EHr7IE4TU2XEj48YNueJnfk8i_pxt9dlyx-V2sToGt92w27MlOT0KxtnbQ7-DPE3ODySgEJbXhkMzilpy_7hHOa9sqhDCdYOvj-bGEN3FBgnPpa2IF7BdtEG5viX1Ns0O6qSVgb/file# [following]
--2021-11-12 02:48:33--  https://uc107c41026598443c4b4b10d15c.dl.dropboxusercontent.com/cd/0/inline/BZwvAd2HPjDU39sa6EHr7IE4TU2XEj48YNueJnfk8i_pxt9dl

In [39]:
# Creamos directorios para la búsqueda por similitud
!mkdir -p /content/data/features_color
!mkdir -p /content/data/features_color/ssearch

!mkdir -p /content/data/features_textura
!mkdir -p /content/data/features_textura/ssearch

In [40]:
# Generamos un archivo catalog.txt que contiene la ruta de las imágenes
%cd /content/data/dataset_atributos

import os

filenames = []
thisdir = os.getcwd()  # current working directory
print(f"directorio actual: {thisdir}")
# r=root, d=directories, f = files
for r, d, f in os.walk(thisdir):
    for file in f:
        if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
            filenames.append(os.path.join(r, file))
        else:
            print(os.path.join(r, file))

print(f"total de imágenes: {len(filenames)}")

# Escribir la lista de imágenes en un archivo de texto para color
catalog_dir = "/content/data/features_color/ssearch"
catalog_filename = os.path.join(catalog_dir, "catalog.txt")

with open(catalog_filename, 'w') as f:
    f.write("\n".join(map(str, filenames)))
print(f"Catálogo guardado en {catalog_filename}")


# Escribir la lista de imágenes en un archivo de texto para textura
catalog_dir = "/content/data/features_textura/ssearch"
catalog_filename = os.path.join(catalog_dir, "catalog.txt")

with open(catalog_filename, 'w') as f:
    f.write("\n".join(map(str, filenames)))

print(f"Catálogo guardado en {catalog_filename}")

/content/data/dataset_atributos
directorio actual: /content/data/dataset_atributos
total de imágenes: 2000
Catálogo guardado en /content/data/features_color/ssearch/catalog.txt
Catálogo guardado en /content/data/features_textura/ssearch/catalog.txt


## Extracción de features

En este paso debemos modificar algunos archivos de configuración. Para esto, usaremos el editor incorporado en Google Colab y luego guardaremos los cambios con Ctrl-S.

### Color

Para extraer los features de color, usaremos la capa ```conv2_block3_out``` de ResNet. Para esto, debemos hacer las siguientes modificaciones:

En el archivo ```/content/visual_attributes/ssearch.py```:

* agregamos el path del repositorio convnet2 (en este tutorial es ```/content/convnet2```)
* en la línea 34, debemos asegurarnos que la variable que se asigna es ```color_layer```.

En el archivo ```/content/visual_attributes/config/resnet50.config```:

* En el parámetro DATA_DIR, usaremos el directorio ```/content/data/features_color``` creado anteriormente.

In [41]:
# Ejecutamos el código
%cd /content/visual_attributes
!git checkout color
!python ssearch.py -config config/resnet50.config -name RESNET -mode compute

/content/visual_attributes
2021-11-12 02:49:05.642264: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 

### Textura

Para extraer los features de textura, usaremos la capa ```conv4_block6_out``` de ResNet. Para esto, debemos hacer las siguientes modificaciones:

En el archivo ```/content/visual_attributes/ssearch.py```:

* agregamos el path del repositorio convnet2 (en este tutorial es ```/content/convnet2```)
* en la línea 34, debemos asegurarnos que la variable que se asigna es ```texture_layer```.

En el archivo ```/content/visual_attributes/config/resnet50.config```:

* En el parámetro DATA_DIR, usaremos el directorio ```/content/data/features_textura``` creado anteriormente.

In [44]:
# Ejecutamos el código
%cd /content/visual_attributes
!git checkout texture
!python ssearch.py -config config/resnet50.config -name RESNET -mode compute

/content/visual_attributes
Fetching origin
error: Your local changes to the following files would be overwritten by checkout:
	config/resnet50.config
	ssearch.py
Please commit your changes or stash them before you switch branches.
Aborting
2021-11-12 03:00:47.837553: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)   

## Búsqueda por similitud

Finalmente, realizaremos una búsqueda por similitud a partir de los features obtenidos. Para esto, debemos modificar los archivos ```/content/visual_attributes/ssearch.py``` y ```/content/visual_attributes/config/resnet50.config``` tal como se indica en el paso anterior, dependiendo de si queremos buscar por color o por textura.

En las figuras a continuación, se muestran 4 búsquedas usando cada atributo (color y textura). La imagen superior izquierda corresponde a la query y las otras son aquellas imágenes más similares dentro del catálogo, según los features escogidos (color o textura).

### Color

In [45]:
%cd /content/visual_attributes/
!git checkout color
!python ssearch.py -config config/resnet50.config -name RESNET -mode search

/content/visual_attributes
Fetching origin
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 2), reused 7 (delta 2), pack-reused 0
Unpacking objects: 100% (7/7), done.
From https://github.com/alvaro-neira/visual_attributes
 * [new branch]      color      -> origin/color
error: Your local changes to the following files would be overwritten by checkout:
	config/resnet50.config
	ssearch.py
Please commit your changes or stash them before you switch branches.
Aborting
2021-11-12 03:04:42.418801: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to               

In [ ]:
from IPython.display import Image, display

display(Image('./25.png_cos_square_root_result.png', width=700, height=700))
display(Image('./flower_1.jpg_cos_square_root_result.png', width=700, height=700))
display(Image('./striped_1.jpeg_cos_square_root_result.png', width=700, height=700))
display(Image('./striped_2.jpeg_cos_square_root_result.png', width=700, height=700))

### Textura

In [ ]:
%cd /content/visual_attributes/
!git checkout texture
!python ssearch.py -config config/resnet50.config -name RESNET -mode search

/content
python3: can't open file 'ssearch.py': [Errno 2] No such file or directory


In [ ]:
display(Image('./25.png_cos_square_root_result.png', width=700, height=700))
display(Image('./flower_1.jpg_cos_square_root_result.png', width=700, height=700))
display(Image('./striped_1.jpeg_cos_square_root_result.png', width=700, height=700))
display(Image('./striped_2.jpeg_cos_square_root_result.png', width=700, height=700))